In [1]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity


import itertools
#sys.path


import regex as re
import datetime
import math
import time
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import plotly
plotly.tools.set_credentials_file(username='juliettapc', api_key='nM6iUdx6dGaOiPXQTwpP')   # go to: https://plot.ly/settings/api#/   for a new key if needed

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [2]:
                                  

####  NEW FILE INCLUDING EARLY CITATIONS OF YOUNG REFERENCES:   ../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit.pkl
                                            
#%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))



%time df_merged = pickle.load(open('../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit_and_after_accretion_time.pkl', 'rb'))
print ("done loading pickles", df_merged.shape)




df_merged = df_merged[df_merged['cite_count'] != -1]   # i dont know why, but there are 7 occurrences with value -1



plos_df = df_merged.drop_duplicates(subset=['paper_UT'])
print (plos_df.shape)



df_merged.head()

CPU times: user 4.08 s, sys: 1.02 s, total: 5.1 s
Wall time: 10 s
done loading pickles (5787634, 34)
(156558, 34)


,occurence,reference_UT,reference_rank,regex_sect_index,cite_count,ref_pub_year,paper_cite_count,plos_pub_year,sect_char_pos,sect_char_total,...,plos_article_type,num_cit_young_ref_by2009,num_cit_young_ref_by2010,num_cit_young_ref_by2011,num_cit_young_ref_by2012,num_cit_young_ref_by2013,num_cit_young_ref_by2009after8,num_cit_young_ref_by2008after8,num_cit_young_ref_by2010after8,num_cit_young_ref_by2007after8
0,1,A1995QY75100004,1,0,60.0,1995.0,2,2013.0,139,4029,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,A1995QY75100004,1,3,60.0,1995.0,2,2013.0,494,5398,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,000263911400006,3,0,5.0,2009.0,2,2013.0,142,4029,...,@ Article,0.0,0.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0
5,1,000289279600018,4,0,29.0,2011.0,2,2013.0,269,4029,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0
6,3,000289279600018,4,3,29.0,2011.0,2,2013.0,3844,5398,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0


In [4]:
#### dictionary category-code
#   ../data/dict_categ_code.pkl



dict_categ_code = pickle.load(open('../data/dict_categ_code.pkl', 'rb'))
#print (len(dict_categ_code))



print ("from the entire df ",df_merged.shape, "  ; and plos one records:",  df_merged[df_merged['plos_j1']== "PLOS ONE"].shape,"\n\n")
# {'Biology and life sciences': 0,
#  'Computer and information sciences': 1,
#  'Earth sciences': 2,
#  'Ecology and environmental sciences': 3,
#  'Engineering and technology': 4,
#  'Medicine and health sciences': 5,
#  'People and places': 6,
#  'Physical sciences': 7,
#  'Research and analysis methods': 8,
#  'Science policy': 9,
#  'Social sciences': 10}

dict_code_categ={}
dict_size_categ={}
for categ in dict_categ_code:
    code = str(dict_categ_code[categ])
    
    df_selection_categ = df_merged[df_merged['categ_codes'].str.contains(code)]
   # print (categ, code, df_selection_categ.shape)
    size= len(df_selection_categ)
    dict_size_categ[size] = categ
    dict_code_categ[code] = categ

for size in reversed(sorted(dict_size_categ)):
    print (size, dict_size_categ[size])

from the entire df  (5787630, 34)   ; and plos one records: (5368040, 34) 


5087018 Biology and life sciences
3807428 Medicine and health sciences
3260169 Research and analysis methods
1766356 Physical sciences
1035007 Computer and information sciences
647228 Social sciences
579580 People and places
526791 Ecology and environmental sciences
444753 Earth sciences
322656 Engineering and technology
36492 Science policy


In [4]:
# ######  i create the new columns for fraction of references in each section (labelled 0 to 7)



# %time plos_df = pickle.load(open('../data/plos_paper_dataframe_ONLY_ARTICLES_num_ref_sect_young_old.pkl', 'rb'))
# print ("done loading plos_df", plos_df.shape)
# plos_df_simplified = plos_df[['paper_UT','num_ref_section0','num_ref_section1','num_ref_section2','num_ref_section3','num_ref_section4','num_ref_section5','num_ref_section6','num_ref_section7',
#     'fract_old_ref_section0', 'fract_old_ref_section1', 'fract_old_ref_section2', 'fract_old_ref_section3', 'fract_old_ref_section4', 'fract_old_ref_section5', 'fract_old_ref_section6', 'fract_old_ref_section7',\
#     'fract_young_ref_section0', 'fract_young_ref_section1', 'fract_young_ref_section2', 'fract_young_ref_section3', 'fract_young_ref_section4', 'fract_young_ref_section5', 'fract_young_ref_section6', 'fract_young_ref_section7']]

# # print ("plos_df_simplified", plos_df_simplified.shape)




# #df_merged = pd.merge(df_merged, plos_df_simplified, on='paper_UT', how='left')
# df_merged = pd.merge(df_merged, plos_df_simplified, on='paper_UT', how='left')
# print ("df_merged", df_merged.shape)

# # df_merged[['paper_UT','reference_UT','regex_sect_index','eff_num_ref','num_ref_section0','num_ref_section1','num_ref_section2','num_ref_section3','num_ref_section4','num_ref_section5','num_ref_section6','num_ref_section7']].head(100)




# df_records_one_ref_per_plos = df_merged.drop_duplicates(subset=['paper_UT'])
# print (df_records_one_ref_per_plos.shape)



# # df_records_one_ref_per_plos.rename(columns={'paper_cite_count_x': 'paper_cite_count', 'plos_pub_year_x': 'plos_pub_year', 'ref_pub_year_x':'ref_pub_year','regex_sect_index_x':'regex_sect_index', 'total_refs_x':'total_refs'}, inplace=True)
# # df_merged.rename(columns={'paper_cite_count_x': 'paper_cite_count', 'plos_pub_year_x': 'plos_pub_year', 'ref_pub_year_x':'ref_pub_year','regex_sect_index_x':'regex_sect_index', 'total_refs_x':'total_refs','reference_UT_x':'reference_UT','cite_count_x':'cite_count'}, inplace=True)


# sorted(df_records_one_ref_per_plos.columns)

# df_records_one_ref_per_plos.head()



# ###OJO!!!!! FALTA POR HACER BIEN EL MERGE PARA EVITAR REPETICION DE COLUMNAS!!!

CPU times: user 442 ms, sys: 88.4 ms, total: 530 ms
Wall time: 529 ms
done loading plos_df (156610, 87)
df_merged (5787630, 58)
(156558, 58)


,occurence,reference_UT,reference_rank,regex_sect_index,cite_count,ref_pub_year,paper_cite_count,plos_pub_year,sect_char_pos,sect_char_total,...,fract_old_ref_section6,fract_old_ref_section7,fract_young_ref_section0,fract_young_ref_section1,fract_young_ref_section2,fract_young_ref_section3,fract_young_ref_section4,fract_young_ref_section5,fract_young_ref_section6,fract_young_ref_section7
0,1,A1995QY75100004,1,0,60.0,1995.0,2,2013.0,139,4029,...,0.0,0.0,0.000000,0.0,0.0,0.032258,0.0,0.0,0.0,0.0
31,1,000088374400026,1,0,477.0,2000.0,2,2015.0,543,2207,...,0.0,0.0,0.023256,0.0,0.0,0.023256,0.0,0.0,0.0,0.0
74,1,000313346100007,2,0,6.0,2013.0,1,2015.0,591,2474,...,0.0,0.0,0.181818,0.0,0.0,0.090909,0.0,0.0,0.0,0.0
85,1,000301376000017,2,0,35.0,2012.0,8,2012.0,762,2700,...,0.0,0.0,0.130435,0.0,0.0,0.086957,0.0,0.0,0.0,0.0
108,1,000232311300030,1,0,1687.0,2005.0,98,2008.0,77,3195,...,0.0,0.0,0.000000,0.0,0.0,0.040000,0.0,0.0,0.0,0.0


In [6]:
### i need this function to do the randomization scheme that preserves groups of references cited together in  a paper

def get_list_lists_references(preselection_df3):

    
    



    ####  NOTA: preselection_df3 only includes one instance of paper_UT-ref_UT  (the first occurrence in each paper), and i sort it too:
    preselection_df3.sort_values(by=['paper_UT','reference_UT'], inplace=True)


    distance_threshold = 5  # characters tops to separate group ref

    list_lists_all_ref = []  # WITH STRUCTURE
    lista_ref = []           # WITHOUT STRUCTURE
    cont = 0
    for paper_UT, group_df in preselection_df3.groupby(['paper_UT']):  #### OJO!!!! THIS LOOP IS WAY FASTER THAN DOING:  for   paper_UT in list_paper_UT    !!!!    

        group_df.sort_values(by=['regex_sect_index','sect_char_pos','reference_UT'],inplace = True)  # i sort the reference of a paper by section first, then by location within the section



        ### first i take care of the isolated references: 
        group_df1= group_df[group_df['isolated_citation'] ==1]        
        for index, row in group_df1.iterrows():  

            ref_UT = row['reference_UT']
            aux = [ref_UT]
            lista_ref.append(ref_UT)   
            list_lists_all_ref.append(aux)




        #### then i take care of the group references:
        group_df0= group_df[group_df['isolated_citation'] == 0]  
        previous_position = 0
        list_group_ref = []
        for index, row in group_df0.iterrows():

            ref_UT = row['reference_UT']
            lista_ref.append(ref_UT)   # list without structure (for comparison reasons)

            position = row['sect_char_pos']

            if previous_position == 0:  # for the very first entry                
                list_group_ref.append(ref_UT)


            else:  # for all other entries
                if (position - previous_position) <= distance_threshold  :   # if the current ref is close to the previous one
                     list_group_ref.append(ref_UT)
                else: 
                    list_lists_all_ref.append(list_group_ref)
                    list_group_ref = []
                    list_group_ref.append(ref_UT)




            previous_position = position

        list_lists_all_ref.append(list_group_ref)  # i need this for the final group/isolated one!!!

        cont +=1   







    #### i flatten out the list of lists for comparison purposes

    flat_list = []    
    for sublist in list_lists_all_ref:
        for item in sublist:
            flat_list.append(item)

    print ("list_lists created:", len(list_lists_all_ref), ' without structure:', len(lista_ref), "   flat_list:", len(flat_list))


    # for i in range(len(flat_list)):
    #     print (lista_ref[i], flat_list[i])
    

    return list_lists_all_ref
    

In [8]:
#######################  MORE SOPHISTICATED RANDOMIZATION SCHEME: ALSO CONTROLING FOR PLOS FIELD  (AS WELL AS PLOS YEAR)  AND PRESERVING REFERENCES THAT ARE CITED TOGETHER IN A GROUP
############################################## ####################### ####################### ####################### ####################### ####################### 


### Figure 4A



#### (i compare the usage of top/nontop references by top/nontop plos papers with a null model that comes from randomizing the data)








#### (i compare the usafe of top/nontop references by top/nontop plos papers with a null model that comes from randomizing the data)

Niter=1000

list_list_years = [[2008],[2009],[2010],[2011],[2012],[2013],[2014]]

#years=[2008]
    
for years in list_list_years:
   

    string_references_age="all"   #young"#old"  # young # all   for the selection of what references i include
    string_isolated_ref=""  #""   # 0  or 1 (or empty string, to include all ref)
    string_self_ref=0    #""#1   # 0  or 1 (or empty string, to include all ref)







    ######### plos ONE categories. 
    string_code_categ="0" #  ojo!!! the codes are strings, not integers. if i want to include multiple subjects:  "1 2 8"

    #  '0': 'Biology and life sciences'             6,032,537
    #  '1': 'Computer and information sciences'     1,207,799
    #  '10': 'Social sciences'                      755,899
    #  '2': 'Earth sciences'                        533,155
    #  '3': 'Ecology and environmental sciences'    624,142
    #  '4': 'Engineering and technology'            382,247 
    #  '5': 'Medicine and health sciences'          4,535,926   
    #  '6': 'People and places'                     691,523
    #  '7': 'Physical sciences'                     2,100,827
    #  '8': 'Research and analysis methods'         3,871,470
    #  '9': 'Science policy'                        43,360 






    # for string_code_categ in range(11):
    #     string_code_categ = str(string_code_categ)









    ######### plos journals 
    string_journal=""

        # PLOS ONE       6,367,070
        # PLOS GENET      149,923
        # PLO NE TR D     138,289   # (neglected tropical diseases)
        # PLOS PATHOG     109,803
        # PLOS COMPUT      77,924
        # PLOS BIOL        56,754
        # PLOS MED         24,506



    ######### WoS subject categories. 
    string_plos_field=""#['D CU BIOLOGY']"

    # ['D RO MULTIDISCIPLINARY SCIENCES']                                                                                                       4464540
    # ['D CU BIOLOGY']                                                                                                                          1055045
    # ['D RO MULTIDISCIPLINARY SCIENCES', 'D CU BIOLOGY']                                                                                        847485
    # ['D KM GENETICS & HEREDITY']                                                                                                               149923
    # ['D YU TROPICAL MEDICINE', 'D TI PARASITOLOGY']                                                                                            138289
    # ['D ZE VIROLOGY', 'D QU MICROBIOLOGY', 'D TI PARASITOLOGY']                                                                                109803
    # ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY']                                                          77687
    # ['D CQ BIOCHEMISTRY & MOLECULAR BIOLOGY', 'D CU BIOLOGY']                                                                                   56754
    # ['D PY MEDICINE, GENERAL & INTERNAL']                                                                                                       24506
    # ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY', 'D PO MATHEMATICS, INTERDISCIPLINARY APPLICATIONS']        237











    print ("original size:",df_merged.shape)








    ##### preselection by plos year
    print (years)
    preselection_df = df_merged[df_merged['plos_pub_year'].isin(years)]  
    print ("size of preselection1 (by plos years):",preselection_df.shape)





    #### i remove self-citations
    if (string_self_ref==0) or  ( string_self_ref == 1 ): 
        preselection_df = preselection_df[preselection_df['self_citation']== string_self_ref ]  
        if string_self_ref ==0:
            string_self_ref = ", no self-cit"
        elif string_self_ref ==1:
            string_self_ref = ", only self-cit"





    ######### preselection by isolated or group references:
    if (string_isolated_ref==0) or  ( string_isolated_ref == 1 ): 
        preselection_df0 = preselection_df[preselection_df['isolated_citation']== string_isolated_ref ]  

        if string_isolated_ref ==0:
            string_isolated_ref = ", group ref"
        elif string_isolated_ref ==1:
            string_isolated_ref = ", isolated ref"
    else:    
        preselection_df0 = preselection_df   
        print ("size of preselection1 (by isolated/group ref):",preselection_df0.shape, string_isolated_ref)








    ######### preselection by plos ONE subject category:
    if string_code_categ=="": 
        preselection_df111 = preselection_df0
    else:    
        if " " not in string_code_categ:  # to include one single category
            preselection_df111 = preselection_df0[preselection_df0['categ_codes'].str.contains(string_code_categ)]        
            string_code_categ = " "+dict_code_categ[string_code_categ]  

        else:  # if multiple codes-categories
            list_codes = string_code_categ.split(" ")
            print (list_codes)

            if len(list_codes) >= 2:              
                preselection_df111 = preselection_df0[ preselection_df0['categ_codes'].str.contains('|'.join(list_codes)) ]  # to look for partial matches from a list of strings!!!!!


            string_code_categ = "" 
            for code in list_codes:
                string_code_categ += "-"+dict_code_categ[code] 


        print (" size of preselection (by plos ONE subject category):",preselection_df111.shape, string_code_categ)








    ######### preselection by plos journal:
    if string_journal=="": 
        preselection_df1 = preselection_df111
    else:    
        preselection_df1 = preselection_df111[preselection_df111['plos_j1']== string_journal ]  
    print (" size of preselection2 (by plos journal):",preselection_df1.shape, string_journal)







    ######### preselection by plos field:
    if string_plos_field=="": 
        preselection_df2 = preselection_df1
    else:    
        preselection_df2 = preselection_df1[preselection_df1['plos_field']== string_plos_field ]  
    print (" size of preselection2 (by plos field):",preselection_df2.shape, string_plos_field)





    ##### preselection only young/old references:       
    preselection_df3 = preselection_df2
    if string_references_age == "young":
        time_window_age = 1   
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] >= (min(years)-time_window_age) ]   

        print ("    size of preselection3 (only young references):",preselection_df3.shape, string_references_age)

    elif string_references_age == "old":
        time_window_age = 10    
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] <= (min(years)-time_window_age) ]   

        print ("    size of preselection3 (only old references):",preselection_df3.shape,string_references_age )





    N_plos = len(preselection_df3.paper_UT.unique())         
    N_ref = len(preselection_df3.reference_UT.unique()) 
    N_all = len(preselection_df3)




    print ("     N plos:", N_plos,"  N  ref:",N_ref, " N records:", N_all)        



    preselection_df3 = preselection_df3.drop_duplicates(subset=['paper_UT', 'reference_UT'])

    print ("OJO!!! EACH REFERENCE ONLY COUNTED ONCE PER PAPER:")

    N_plos = len(preselection_df3.paper_UT.unique())         
    N_ref = len(preselection_df3.reference_UT.unique()) 
    N_all = len(preselection_df3)




    print ("     N plos:", N_plos,"  N  ref:",N_ref, " N records:", N_all)        














    ############## i define quantiles for plos papers (for that subselection, and based on their FINAL number of citations):
    list_q_plos=[.1,.9,1]
    #list_q_plos=[.05,.95,1]

    df_for_quantiles_plos = preselection_df3.drop_duplicates(subset=['paper_UT'])   # ojo!!! dont use preselection_df3 directly because there are REPETITIONS!!!!

    quantiles=sorted(list(df_for_quantiles_plos['paper_cite_count'].quantile(list_q_plos).to_dict().items())) #mean 10.68 

    print ("\n\ncitation bins for the selected plos:", list_q_plos)#,quantiles, df_for_quantiles_plos.shape)   

    lista_bins_plos=[]
    old_value=0
    for item in quantiles:
        pair=[old_value, int(item[1])]
        lista_bins_plos.append(pair)
        old_value = int(item[1])

    #print (lista_bins_plos, min(preselection_df3['paper_cite_count']), max(preselection_df3['paper_cite_count']))

    print ("\nbins for PLOS papers:")

    cont = 0
    dict_bin_list_plos_UT={}
    for item in lista_bins_plos:

        minimo = item[0]
        maximo = item[1]   

        df_select = preselection_df3[(preselection_df3['paper_cite_count'] >= minimo)  &  (preselection_df3['paper_cite_count'] < maximo)]
        llave=str(minimo)+"-"+str(maximo)
        dict_bin_list_plos_UT[llave]= list(df_select.paper_UT.unique())
        print (" ",llave, "  N:",len(list(df_select.reference_UT.unique())), "  avg # ref:",df_select.drop_duplicates(subset=['paper_UT']).total_refs.mean())
        max_key_plos=llave


        if cont ==0:
            min_key_plos = llave
            string_range_bottom_plos =   "["+llave+"] citations"


        cont  +=1



        string_range_top_plos =  "["+llave+"] citations"  

    print ("string for bottom:", string_range_bottom_plos, " string for top:", string_range_top_plos)




    ########## i define quantiles for references (based on their FINAL number of citations)
    #list_q_ref=[.2,.8,1]
    list_q_ref=[.1,.9,1]

    #list_q_ref=[.05,.95,1]
    df_for_quantiles_ref = preselection_df3.drop_duplicates(subset=['reference_UT'])   # ojo!!! remember to remove REPETITIONS!!!!
    quantiles=sorted(list(df_for_quantiles_ref['cite_count'].quantile(list_q_ref).to_dict().items())) #mean 10.68 

    print ("\n\ncitation bins for the references in the selected plos:", list_q_ref,quantiles)    

    lista_bins=[]
    old_value=0
    for item in quantiles:
        pair=[old_value, int(item[1])]
        lista_bins.append(pair)
        old_value = int(item[1])




    print ("\nbins for refrences:")


    cont = 0
    dict_bin_list_ref_UT={}
    for item in lista_bins:

        minimo = item[0]
        maximo = item[1]    

        df_select = preselection_df3[(preselection_df3['cite_count'] >= minimo)  &  (preselection_df3['cite_count'] < maximo)]
        llave=str(minimo)+"-"+str(maximo)
        dict_bin_list_ref_UT[llave]=list(df_select.reference_UT.unique())
        print (" ",llave, "N:",len(list(df_select.reference_UT.unique())), "  avg # ref:",df_select.drop_duplicates(subset=['reference_UT']).total_refs.mean())
        max_key_ref=llave

        if cont ==0:
            min_key_ref = llave
        cont  +=1




    ############### i create the list of top plos, top ref, bottom plos and bottom ref:
    #########################

    lista_top_plos = dict_bin_list_plos_UT[max_key_plos]
    print ("\n\n# UTs top",(100-100*list_q_plos[-2]),"% plos:",len(lista_top_plos))

    lista_top_ref=dict_bin_list_ref_UT[max_key_ref]
    print ("# UTs top",(100-100*list_q_ref[-2]),"% ref:", len(lista_top_ref))


    lista_bottom_plos = dict_bin_list_plos_UT[min_key_plos]
    print ("# UTs bottom ",(100*list_q_plos[0]),"% plos:",len(lista_bottom_plos))

    lista_bottom_ref=dict_bin_list_ref_UT[min_key_ref]
    print ("# UTs bottom ",(100*list_q_ref[0]),"% ref:", len(lista_bottom_ref))

    list_plos_in_year= list(preselection_df3.paper_UT.unique())
    print ("Tot # records:",len(preselection_df3),", # plos:",len(list_plos_in_year))





    ######  i look at the usage of the top ref
    ################################################  

    df_top_ref = preselection_df3[preselection_df3['reference_UT'].isin(lista_top_ref)]


    df_top_ref_top_plos = df_top_ref[df_top_ref['paper_UT'].isin(lista_top_plos)]
    df_top_ref_bottom_plos = df_top_ref[df_top_ref['paper_UT'].isin(lista_bottom_plos)]


    usage_top_ref_top_plos = len(df_top_ref_top_plos)/float(len(df_top_ref))
    usage_top_ref_bottom_plos = len(df_top_ref_bottom_plos)/float(len(df_top_ref))


    print ("fraction of usage of top ref by ")
    print ("  top",(100-100*list_q_plos[-2]),"% plos:",  usage_top_ref_top_plos)
    print ("  bottom",(100*list_q_plos[0]),"% plos:",usage_top_ref_bottom_plos  )







    ######  i look at the usage of the non-top ref
    ################################################      

    df_non_top_ref = preselection_df3[preselection_df3['reference_UT'].isin(lista_bottom_ref)]


    df_non_top_ref_top_plos = df_non_top_ref[df_non_top_ref['paper_UT'].isin(lista_top_plos)]
    df_non_top_ref_bottom_plos = df_non_top_ref[df_non_top_ref['paper_UT'].isin(lista_bottom_plos)]

    usage_non_top_ref_top_plos = len(df_non_top_ref_top_plos)/float(len(df_non_top_ref))
    usage_non_top_ref_bottom_plos = len(df_non_top_ref_bottom_plos)/float(len(df_non_top_ref))


    print ("fraction of usage of non-top ref by ")
    print ("  top",(100-100*list_q_plos[-2]),"% plos:", usage_non_top_ref_top_plos )
    print ("  bottom",(100*list_q_plos[0]),"% plos:", usage_non_top_ref_bottom_plos )






    ####################
    # I canculate the null model (usage of references by top and non top plos papers, from the randomized data)
    #################################################  

    lista_usage_top_ref_by_top_plos_rand = []
    lista_usage_top_ref_by_bottom_plos_rand = []

    lista_usage_nontop_ref_by_top_plos_rand = []
    lista_usage_nontop_ref_by_bottom_plos_rand = []





    #### first i get the list of lists corresponding to the references used in the selected df, preserving the reference grouping or isolation:  
    lista_lists_values = get_list_lists_references(preselection_df3)

    print ("len list_lists_all_ref:",len(lista_lists_values), preselection_df3.shape)


    for i in range(Niter):

        print (i)



        #### old, simple randomization scheme (only controling for year)
    #     lista_values = list(preselection_df3.reference_UT)   #[i for i in range (len(df_merged))]
    #     random.shuffle(lista_values)
    #     preselection_df3['randomized_ref_UT'] = lista_values






        ########   new randomization scheme (controling for year, but also preserving groups of references cited together in a paper):   
        #### lista_values is created outside the Niter loop   (i only need to do it once per selected df)    



        random.shuffle(lista_lists_values)      
        ### to flat out a list of lists:   
        flat_list = []    
        for sublist in lista_lists_values:
            for item in sublist:
                flat_list.append(item)


       # print ("len flat list:", len(flat_list), preselection_df3.shape)

        preselection_df3['randomized_ref_UT'] = flat_list   ### ojo!!! esto randomiza paper_UT, no reference_UT  !!!!!!








        ####### (RANDOMIZED)  i look at the usage of the top ref
        df_top_ref_rand = preselection_df3[preselection_df3['randomized_ref_UT'].isin(lista_top_ref)]

        df_top_ref_top_plos_rand = df_top_ref_rand[df_top_ref_rand['paper_UT'].isin(lista_top_plos)]
        df_top_ref_bottom_plos_rand = df_top_ref_rand[df_top_ref_rand['paper_UT'].isin(lista_bottom_plos)]


        usage_top_ref_top_plos_rand = len(df_top_ref_top_plos_rand)/float(len(df_top_ref_rand))
        usage_top_ref_bottom_plos_rand = len(df_top_ref_bottom_plos_rand)/float(len(df_top_ref_rand))


        lista_usage_top_ref_by_top_plos_rand.append(usage_top_ref_top_plos_rand)
        lista_usage_top_ref_by_bottom_plos_rand.append(usage_top_ref_bottom_plos_rand)






        #######  (RANDOMIZED) i look at the usage of the non-top ref            
        df_non_top_ref_rand = preselection_df3[preselection_df3['randomized_ref_UT'].isin(lista_bottom_ref)]        

        df_non_top_ref_top_plos_rand = df_non_top_ref_rand[df_non_top_ref_rand['paper_UT'].isin(lista_top_plos)]
        df_non_top_ref_bottom_plos_rand = df_non_top_ref_rand[df_non_top_ref_rand['paper_UT'].isin(lista_bottom_plos)]


        usage_non_top_ref_top_plos_rand = len(df_non_top_ref_top_plos_rand)/float(len(df_non_top_ref_rand))
        usage_non_top_ref_bottom_plos_rand = len(df_non_top_ref_bottom_plos_rand)/float(len(df_non_top_ref_rand))


        lista_usage_nontop_ref_by_top_plos_rand.append(usage_non_top_ref_top_plos_rand)
        lista_usage_nontop_ref_by_bottom_plos_rand.append(usage_non_top_ref_bottom_plos_rand)




    print("plos category:", string_code_categ)




    print ("\n\n\n\navg randomized!!")
    print("fraction of usage of top ref by")
    print ("  top",(100-100*list_q_plos[-2]),"% plos:",  np.mean(lista_usage_top_ref_by_top_plos_rand) )   
    print ("  bottom",(100*list_q_plos[0]),"% plos:",np.mean(lista_usage_top_ref_by_bottom_plos_rand)  )




    print ("\n\navg randomized")
    print ("fraction of usage of non-top ref by ")
    print ("  top",(100-100*list_q_plos[-2]),"% plos:", np.mean(lista_usage_nontop_ref_by_top_plos_rand) )   
    print ("  bottom",(100*list_q_plos[0]),"% plos:", np.mean(lista_usage_nontop_ref_by_bottom_plos_rand) ,"\n\n\n")












    #############################  second part


    ###  PLOT FIGURE FOR MORE SOPHISTICATED VERSION OF THE RANDOMIZATION SCHEME
    ######     (RUN PREVIOUS CELL FIRST, TO GET THE BOOT-STRAPPING DATA)


    ### group by Top PApers  Bottom Papers




    #########################################

    #lista_bin_names=['Top '+str(int(100-100*list_q_plos[-2]))+'%<br>papers', 'Bottom '+str(int(list_q_plos[0]*100))+'%<br>papers']
    #lista_bin_names = ['Bottom '+str(int(list_q_plos[0]*100))+'%<br>papers', 'Top '+str(int(100-100*list_q_plos[-2]))+'%<br>papers']

    lista_bin_names = ['Bottom '+str(int(list_q_plos[0]*100))+'% papers<br>'+string_range_bottom_plos, 'Top '+str(int(100-100*list_q_plos[-2]))+'% papers<br>'+string_range_top_plos]






    lista_for_top_ref = [ usage_top_ref_top_plos, usage_top_ref_bottom_plos]
    lista_for_bottom_ref = [usage_non_top_ref_top_plos, usage_non_top_ref_bottom_plos]




    lista_for_top_ref = [  usage_top_ref_bottom_plos, usage_top_ref_top_plos]
    lista_for_bottom_ref = [usage_non_top_ref_bottom_plos, usage_non_top_ref_top_plos]





    ###### this is the null model 


    # lista_expectations_top_ref = [np.mean(lista_usage_top_ref_by_top_plos_rand),np.mean(lista_usage_top_ref_by_bottom_plos_rand)]  
    # lista_expectations_bottom_ref = [np.mean(lista_usage_nontop_ref_by_top_plos_rand), np.mean(lista_usage_nontop_ref_by_bottom_plos_rand)] 


    lista_expectations_top_ref = [np.mean(lista_usage_top_ref_by_bottom_plos_rand), np.mean(lista_usage_top_ref_by_top_plos_rand)]  
    lista_expectations_bottom_ref = [ np.mean(lista_usage_nontop_ref_by_bottom_plos_rand),np.mean(lista_usage_nontop_ref_by_top_plos_rand)] 






    # list_errors_top_ref = [2.*np.std(lista_usage_top_ref_by_top_plos_rand), 2.*np.std(lista_usage_top_ref_by_bottom_plos_rand)] 
    # list_errors_bottom_ref = [2.*np.std(lista_usage_nontop_ref_by_top_plos_rand) , 2.*np.std(lista_usage_nontop_ref_by_bottom_plos_rand)] 


    list_errors_top_ref = [2.*np.std(lista_usage_top_ref_by_bottom_plos_rand), 2.*np.std(lista_usage_top_ref_by_top_plos_rand) ] 
    list_errors_bottom_ref = [2.*np.std(lista_usage_nontop_ref_by_bottom_plos_rand), 2.*np.std(lista_usage_nontop_ref_by_top_plos_rand) ] 






    z_score_top_ref_by_top_plos = (usage_top_ref_top_plos - np.mean(lista_usage_top_ref_by_top_plos_rand))/np.std(lista_usage_top_ref_by_top_plos_rand)
    z_score_nontop_ref_by_top_plos = (usage_non_top_ref_top_plos - np.mean(lista_usage_nontop_ref_by_top_plos_rand))/np.std(lista_usage_nontop_ref_by_top_plos_rand)

    z_score_top_ref_by_bottom_plos = (usage_top_ref_bottom_plos - np.mean(lista_usage_top_ref_by_bottom_plos_rand))/np.std(lista_usage_top_ref_by_bottom_plos_rand)
    z_score_nontop_ref_by_bottom_plos = (usage_non_top_ref_bottom_plos - np.mean(lista_usage_nontop_ref_by_bottom_plos_rand))/np.std(lista_usage_nontop_ref_by_bottom_plos_rand)




    print ('zscore top ref cited  by top plos:', z_score_top_ref_by_top_plos)
    print ('zscore bottom ref cited  by top plos:', z_score_nontop_ref_by_top_plos)

    print ('zscore top ref cited  by bottom plos:', z_score_top_ref_by_bottom_plos)
    print ('zscore bottom ref cited  by bottom plos:', z_score_nontop_ref_by_bottom_plos)



    title_string=''#'s top-ref by top plos: '+str(z_score_top_ref_by_top_plos)+';  zs top-ref by bottom plos: '+str(z_score_top_ref_by_bottom_plos)+\
    #'<br>zs nontop-ref by top plos: '+str(z_score_nontop_ref_by_top_plos)+';  zs nontop-ref by bottom plos: '+str(z_score_nontop_ref_by_bottom_plos)



    size_bar_name = 30#45
    y_pos_bar_names = -.039
    angle = -70

    trace1 = go.Bar(
        x=lista_bin_names,
        y=lista_for_top_ref,
    #     text=['by top '+str(int(100-100*list_q_plos[-2]))+'% papers', 'by top '+str(int(100-100*list_q_plos[-2]))+'% papers'],    
    #     name='by top '+str(int(100-100*list_q_plos[-2]))+'% papers',   
        marker=dict(
            color='#88419d',           
        ),



    )



    trace2 = go.Bar(
        x=lista_bin_names,
        y=lista_expectations_top_ref,
    #     name='   expected value',
    #     text=['Expected value for top', 'Expected value for top'],  
        error_y=dict(
           # type='data',
            array=list_errors_top_ref,
            thickness=5,
            visible=True
        ),
        marker=dict(
            color='#c994c7',         
        ),
    )


    trace3 = go.Bar(
        x=lista_bin_names,
        y=lista_for_bottom_ref,
    #     name='by bottom '+str(int(list_q_plos[0]*100))+'% papers', 
    #     text=['by bottom '+str(int(list_q_plos[0]*100))+'% papers','by bottom '+str(int(list_q_plos[0]*100))+'% papers'],
        marker=dict(
            color='#225ea8',   
        ),
    )


    trace4 = go.Bar(
        x=lista_bin_names,
        y=lista_expectations_bottom_ref,
      #  name='   expected value',

        error_y=dict(       
            array=list_errors_bottom_ref,#[0.5, 1, 2],
            thickness=5,
            visible=True
        ),
        marker=dict(
            color='#a6bddb',     
        ),
    )





    data = [trace1, trace2, trace3, trace4]
    layout = go.Layout(   
        title=title_string,
        xaxis = dict(
            side= 'top',
            range = [-.5,1.5],
           # showline =  True,
            #title= 'Plos Citation percentile'),
        ),
        yaxis = dict(
            title= 'Fraction references cited',
            range = [-.07,0.17],
            tickvals=[0.0,0.05,0.1,0.15],
            #showline =  True,
             ),

        showlegend=False,
    #     legend=dict(x=0.75, y=1.05,                 
    #                font=dict(
    #                     #family='sans-serif',
    #                     size=40,
    #                     #color='#000'
    #                     ),
    #                 ),


    #     barmode='stacked',#group',
       # bargap=0.2,
        bargroupgap=0.15,

        annotations = [  
            # the four bars on the left
            dict(
              x = -.34,
              y = y_pos_bar_names,
              text = 'Top '+str(int(100-100*list_q_ref[-2]))+'%<br>references',
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),    
            dict(
              x = -0.14,
              y = y_pos_bar_names,
              text = 'Null Model',
               textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = .08,
              y = y_pos_bar_names,
              showarrow = False,
              text = 'Bottom '+str(int(list_q_ref[0]*100))+'%<br>references', 
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = .28,
              y = y_pos_bar_names,
              text =  'Null Model',
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),



            # the four bars on the right
            dict(  
              x = .68,
              y = y_pos_bar_names,
              text = 'Top '+str(int(100-100*list_q_ref[-2]))+'%<br>references',
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = .88,
              y = y_pos_bar_names,
              text = 'Null Model',
               textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = 1.08,
              y = y_pos_bar_names,
              text = 'Bottom '+str(int(list_q_ref[0]*100))+'%<br>references', 
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = 1.28,
              y = y_pos_bar_names,
              text =  'Null Model',
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),


            ],    
    )








    fig = go.Figure(data=data, layout=layout)




    font_gral=55  # 20 if i wanna see it on the browser, 40 if i care about the png output
    fig['layout']['font']['size'] = font_gral-5


    # Altering x axis
    #fig['layout']['xaxis']['tickfont']['family'] = 'Gill Sans MT'
    fig['layout']['xaxis']['tickangle'] = 0
    # fig['layout']['yaxis']['tickangle'] = -90
    # fig['layout']['xaxis']['titlefont']['size'] = font_gral -10
    # fig['layout']['yaxis']['titlefont']['size'] = font_gral -10

    fig['layout']['xaxis']['tickfont']['size'] = font_gral -5
    fig['layout']['yaxis']['tickfont']['size'] = font_gral -10




    fig['layout']['margin']=dict(
            l=300,
           # r=50,
            b=100,
            t=200,
            pad=15
        )




    py.iplot(fig, filename='grouped-bar')


    fig_filename='fract_usage_top_bottom_ref_'+str(years[0])+string_code_categ.replace(" ","_")
    offline.plot(fig, auto_open=True, image = 'png', image_filename='../plots/'+fig_filename ,image_width=2000, image_height=1200, filename='../plots/'+fig_filename+'.html', validate=True)





    # earth science, 2013, 1000 iter, simple randomization scheme:

    # zscore top ref cited  by top plos: 1.8633822051
    # zscore bottom ref cited  by top plos: -8.29803601221
    # zscore top ref cited  by bottom plos: -4.18320566755
    # zscore bottom ref cited  by bottom plos: 2.22458205576





    ## earth science, 2013, 1000 iter, more sophisticated randomization scheme (preserving reference groups/isolated references):

    # zscore top ref cited  by top plos: 1.61069883113
    # zscore bottom ref cited  by top plos: -8.02168733553
    # zscore top ref cited  by bottom plos: -3.84738913187
    # zscore bottom ref cited  by bottom plos: 2.16556097748


original size: (5787630, 34)
[2008]
size of preselection1 (by plos years): (104056, 34)
size of preselection1 (by isolated/group ref): (104056, 34) 
 size of preselection (by plos ONE subject category): (95977, 34)  Biology and life sciences
 size of preselection2 (by plos journal): (95977, 34) 
 size of preselection2 (by plos field): (95977, 34) 
     N plos: 2687   N  ref: 69604  N records: 95977
OJO!!! EACH REFERENCE ONLY COUNTED ONCE PER PAPER:
     N plos: 2687   N  ref: 69604  N records: 79000


citation bins for the selected plos: [0.1, 0.9, 1]

bins for PLOS papers:
  0-7   N: 6060   avg # ref: 38.58
  7-76   N: 57481   avg # ref: 45.280369515
  76-787   N: 8569   avg # ref: 60.3948339483
string for bottom: [0-7] citations  string for top: [76-787] citations


citation bins for the references in the selected plos: [0.1, 0.9, 1] [(0.1, 22.0), (0.9, 549.0), (1.0, 326393.0)]

bins for refrences:
  0-22 N: 6664   avg # ref: 53.6332533013
  22-549 N: 55966   avg # ref: 54.1429439302

fraction of usage of top ref by 
  top 10.0 % plos: 0.1307634164777022
  bottom 10.0 % plos: 0.07369614512471655
fraction of usage of non-top ref by 
  top 10.0 % plos: 0.07484619553090434
  bottom 10.0 % plos: 0.09881737910809346


KeyboardInterrupt: 

[[0, 2], [2, 8], [8, 25], [25, 79], [79, 1173]] (156558, 58)
[0, 2] (156558, 58) (35805, 58) [[(0.25, 0.07407407407407407), (0.5, 0.14705882352941177), (0.75, 0.23809523809523808)], [(0.25, 0.0), (0.5, 0.04395604395604396), (0.75, 0.0975609756097561)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.02857142857142857)], [(0.25, 0.029411764705882353), (0.5, 0.11363636363636363), (0.75, 0.20930232558139536)]]
[2, 8] (156558, 58) (57980, 58) [[(0.25, 0.06521739130434782), (0.5, 0.13043478260869565), (0.75, 0.2127659574468085)], [(0.25, 0.0), (0.5, 0.04081632653061224), (0.75, 0.08695652173913043)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.0425531914893617)], [(0.25, 0.03333333333333333), (0.5, 0.10714285714285714), (0.75, 0.1935483870967742)]]
[8, 25] (156558, 58) (46718, 58) [[(0.25, 0.04838709677419355), (0.5, 0.10416666666666667), (0.75, 0.17777777777777778)], [(0.25, 0.0), (0.5, 0.03571428571428571), (0.75, 0.07692307692307693)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.04878048780487805)], [(0.25, 0.02564102

'file:///home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/notebooks/testing_annotated_heatmap_color.html'